<a href="https://colab.research.google.com/github/mirceachira/licenta/blob/main/licenta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# !pip install git+https://github.com/trsvchn/calabar.git
# !pip3 install backtrader --user
# !pip3 install pyfolio --user
# !pip3 install tqdm --user
# !pip3 install wheel --user
# !pip3 install pandas --user

In [22]:
import datetime

# Filesystem location
MOUNT_LOCATION = '/content/drive' # When running in google drive
MOUNT_LOCATION = '/root/Projects/licenta/mount/'
DATA_LOCATION = f'{MOUNT_LOCATION}/MyDrive/licenta' # When running in google drive
DATA_LOCATION = f'{MOUNT_LOCATION}' # When running in google drive

TICKERS_LOCATION = f'{DATA_LOCATION}/tickers'
RESULTS_LOCATION = f'{DATA_LOCATION}/results'

# Email notifications
MY_EMAIL = 'chira.mircea.darius@gmail.com'
NOTIFICATION_RECEIVER_LIST = [MY_EMAIL]

# Finance API
YAHOO_FINANCE_MAX_URL = 'https://query1.finance.yahoo.com/v7/finance/download/{TICKER_NAME}?period1=946688400&period2=1609459199&interval=1d&events=history&includeAdjustedClose=true'
# 1609459199 - 2020.12.31 11:59:59 PM GMT
# 946688400 - 2000.01.01 01:00:00 AM GMT
# Tickers
TICKER_NAMES = {
    # Gaming companies
    'ATVI': 'Activision Blizzard, Inc.',
    'EA': 'Electronic Arts Inc.',
    'NTDOY': 'Nintento (traded in US)',

    # Retail
    'GME': 'GameStop Corp.',

    # Tech
    'GOOG': 'Alphabet Inc.',
}
TICKERS = list(TICKER_NAMES.keys())

# Strategy defaults
DEFAULT_FROM_DATE = datetime.datetime(2018, 1, 1)
DEFAULT_TO_DATE = datetime.datetime(2020, 12, 31)
DEFAULT_CASH = 1000.0
DEFAULT_COMMISION = 0.0
DEFAULT_CPU_COUNT = 10

# Some other configs
KLASS_KEY = 'klass'

In [2]:
# Required only when running in google drive
from google.colab import drive

drive.mount(MOUNT_LOCATION)

ModuleNotFoundError: No module named 'google'

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
def add_entry_to_csv(csv_filename, trade_info_dict):
    csv_path = f'{RESULTS_LOCATION}/{csv_filename}.csv'

    # if os.path.exists(csv_path):
    #     csv_df = pd.read_csv(csv_path)

    trade_info_dict = {k: [v] for k, v in trade_info_dict.items()}

    trade_df = pd.DataFrame.from_dict(trade_info_dict)
    trade_df.to_csv(csv_path, mode='a', index=False, header=False)
    # csv_df.append(trade_df)

    # csv_df.to_csv(csv_path)

In [5]:
import pandas as pd


def get_ticker_csv_path(ticker_name):
  return f'{TICKERS_LOCATION}/{ticker_name}.csv'


def get_ticker_csv_as_df(ticker_name):
    return pd.read_csv(get_ticker_csv_path(ticker_name))

In [ ]:
import requests


def download_latest_ticker_csv(ticker_name):
    yahoo_url = YAHOO_FINANCE_MAX_URL.format(TICKER_NAME=ticker_name)
    file_respone = requests.get(yahoo_url)

    csv_path = get_ticker_csv_path(ticker_name)

    print(f'Downloading to {csv_path} about {len(file_respone.content)} bytes from {yahoo_url}')
    with open(csv_path, 'wb') as f:
        f.write(file_respone.content)


def update_all_ticker_csvs():
    for ticker_name in TICKERS:
        download_latest_ticker_csv(ticker_name)


update_all_ticker_csvs()
# download_latest_ticker_csv('GOOG')

In [13]:
import pyfolio as pf
import backtrader as bt


def bt_opt_callback(cb):
    pbar.update()

def test_strategy(strategy_class, ticker_name, from_date=DEFAULT_FROM_DATE, to_date=DEFAULT_TO_DATE, cash=DEFAULT_CASH, commision=DEFAULT_COMMISION, cpu_count=DEFAULT_CPU_COUNT, **strategy_kwargs):
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    kwargs = {
        'ticker': ticker_name,
        **strategy_kwargs
    }

    # Add a strategy
    strats = cerebro.optstrategy(
        strategy_class,
        **kwargs
      )

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=get_ticker_csv_path(ticker_name),
        # Do not pass values before this date
        fromdate=from_date,
        # Do not pass values before this date
        todate=to_date,
        # Do not pass values after this date
        reverse=False
    )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(cash)

    # Add pyfolio analyzer for stats
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # Set the commission
    cerebro.broker.setcommission(commission=commision)

    cerebro.optcallback(cb=bt_opt_callback)

    # Run over everything
    results = cerebro.run(maxcpus=cpu_count)
    
    # # Use this to benchmark final results as well!
    # for result in results:
    #     strat = result[0]
    #     pyfoliozer = strat.analyzers.getbyname('pyfolio')

    #     # TODO: A lot more can be generated using pyfolio but I just couldn't fix it to generate the entire spreadsheet report: https://www.backtrader.com/docu/analyzers/pyfolio/
    #     # gross_lev is no longer needed according to backtrader
    #     returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

    #     yield returns, positions, transactions


def run_through_tickers(strategy_class, ticker_list, *args, **kwargs):
    for ticker_name in ticker_list:
        test_strategy(strategy_class, ticker_name, *args, **kwargs)

        # # Use this to benchmark final results as well!
        # for returns, positions, transactions  in test_strategy(strategy_class, ticker_name, *args, **kwargs):
        #   yield returns, positions, transactions


In [17]:
import pyfolio as pf 


def run_backtest_for_strategy_by_name(strategy_name, ticker_list=TICKERS):
    print(f'Running `run_backtest_for_strategy_by_name` for {strategy_name} for {len(ticker_list)} tickers')
    strategy_setup = copy.deepcopy(STRATEGIES_ALL_CONFIGS)[strategy_name]
    strategy_klass = strategy_setup.pop(KLASS_KEY)
    return run_backtest_for_strategy(strategy_klass, ticker_list, strategy_setup)
    

def list_configs(configs):
    """Translate a configuration with ranges into a list of dicts with value pairs"""
    config_dict_list = [{'_dummy_param': 1}]
    
    for name, value in configs.items():
        if type(value) == range:
            new_config_dict_list = []
            for v in value:
                for cd in config_dict_list:
                   new_config_dict_list.append({
                       name: v,
                       **cd
                   })
            config_dict_list = new_config_dict_list
        else:
            new_config_dict_list = []
            for cd in config_dict_list:
                new_config_dict_list.append({
                    name: value,
                    **cd
                })
            config_dict_list = new_config_dict_list
        
    for x in config_dict_list:
        x.pop('_dummy_param')

    return config_dict_list


# def backtest_job(strategy_klass, ticker, cfg):
#     run_through_tickers(strategy_klass, [ticker], **cfg)

#   for returns, positions, transactions in run_through_tickers(strategy_klass, [ticker], **cfg):
#     ## Awesome if it worked..
#     # pf.create_round_trip_tear_sheet(returns, positions, transactions)
#     # pf.create_returns_tear_sheet(returns)
    
#     ## If you want to show just one outcome at a time
#     # pf.create_simple_tear_sheet(returns)

#     stats_series = pf.timeseries.perf_stats(
#         returns,
#         positions=positions,
#         transactions=transactions
#     )

#     # When benchmarking final results, use this instead + the bad way of running samples?
#     # # Enrich the series
#     # for k, v in config_combination.items():
#     #     stats_series[k] = v

#     # stats_series['ticker'] = ticker

#     return stats_series.copy()


def run_backtest_for_strategy(strategy_klass, ticker_list, configs):
    # df_stuff = []

    config_combination_list = list_configs(copy.deepcopy(configs))

    total_nr_jobs = len(ticker_list) * len(config_combination_list)


    run_through_tickers(strategy_klass, ticker_list, **configs)
        
    # for ticker in ticker_list:
        # cfg = {
        #     'ticker': ticker,
        #     **config_combination
        # }

        # backtest_result = backtest_job(strategy_klass, ticker, configs)

        # df_stuff.append(backtest_result)

        # pbar.update(1)

        # resulting_all_df = pd.concat(df_stuff, axis=1)

    # return resulting_all_df


def run_backtest_for_strategy_for_all_tickers(strategy_klass, configs):
    return run_backtest_for_strategy(strategy_klass, TICKERS, configs)

In [18]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import uuid
import copy

# Import the backtrader platform
import backtrader as bt

In [19]:
class RaynerTeoStrategy(bt.Strategy):
    """
    Rayner Teo Strategy with some additional logging

    Market:
      any stock

    Define the trend:
      (closing?) price above the 200-day moving average

    Entry:
      10-period RSI below 30 (buy on the next day's open)

    Exit:
      10-period RSI above 40, or after 10 trading days (sell on the next day's open)
    """
    params = (
            # TODO: these should be in a defaults class or something maybe
            # SMA
            ('maperiod', 15),

            # RSI
            ('rsi_open_period', 10),
            ('rsi_close_period', 30),
            
            # ADX
            ('adx_period', 14),
            
            # PPO
            ('ppo_period_short', 12), 
            ('ppo_period_long', 26),

            # Stochastic
            ('stochastic_period', 14),

            # Other
            ('days_ago_close_period', 10),
            ('printlog', False),
            ('ticker', 'GME'),
        )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0],
            period=self.params.maperiod
        )
        self.rsi = bt.indicators.RSI(period=self.params.rsi_open_period)
        self.adx = bt.indicators.ADX(period=self.params.adx_period)
        self.ppo = bt.indicators.PPO(
            period1=self.params.ppo_period_short,
            period2=self.params.ppo_period_long
        ) # , period_signal=?)
        self.stochastic = bt.indicators.Stochastic(period=self.params.stochastic_period)
          # TODO: There are other parameters here, add them all or as needed!

        self.order_placed_days_ago = 0
        self.csv_filename = 'main'  # TODO: differentiate between runs without duplication
        # f'{self.params.ticker}_{datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}'

    def notify_order(self, order):
        # Doing: log into a csv:
        #   * all metrics
        #   * stock price
        #   * params at buy, result at close

        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0] and self.rsi[0] < 30:
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                self.trade_info_dict = {
                    'uid': str(uuid.uuid1()),
                    'ticker': self.params.ticker,
                    'date': str(datetime.datetime.now()),
                    'price_open': self.dataclose[0],
                    'adx': self.adx[0],
                    'ppo': self.ppo[0],
                    'stochastic': self.stochastic[0],
                    'maperiod': self.params.maperiod,
                    'rsi_open_period': self.params.rsi_open_period,
                    'rsi_close_period': self.params.rsi_close_period,
                    'adx_period': self.params.adx_period,
                    'ppo_period_short': self.params.ppo_period_short,
                    'ppo_period_long': self.params.ppo_period_long,
                    'stochastic_period': self.params.stochastic_period,
                    'days_ago_close_period': self.params.days_ago_close_period,
                }

                # CONSIDERATION: the 'checked' price vs the 'order accepted at' price 

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
        else:
            if self.rsi[0] > self.params.rsi_close_period or self.order_placed_days_ago == self.params.days_ago_close_period:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

                # Log the selling price
                self.trade_info_dict['price_sell'] = self.dataclose[0]
                add_entry_to_csv(csv_filename=self.csv_filename, trade_info_dict=copy.deepcopy(self.trade_info_dict))

                self.order_placed_days_ago = 0
            else:
                self.order_placed_days_ago += 1


    def stop(self):
        self.log(f'(MA Period {self.params.maperiod}, RSI open {self.params.rsi_open_period}, RSI close {self.params.rsi_close_period}, Close after {self.params.days_ago_close_period} days) Ending Value {self.broker.getvalue()}', doprint=False)

In [24]:
STRATEGIES_ALL_CONFIGS = {
    'Rayner Teo High Winrate': {
        KLASS_KEY: RaynerTeoStrategy,

        # Optimization
        'maperiod': range(195, 205),
        'rsi_open_period': range(7, 14),
        'rsi_close_period': range(27, 35),
        'days_ago_close_period': range(7, 13)
        
        # # Dummy run
        # 'maperiod': 256,
        # 'rsi_open_period': 8,
        # 'rsi_close_period': 32,
        # 'days_ago_close_period': 8
    },
}

In [25]:
from tqdm.auto import tqdm

strategy_name = 'Rayner Teo High Winrate'

cfg_cpy = copy.deepcopy(STRATEGIES_ALL_CONFIGS[strategy_name])
cfg_cpy.pop(KLASS_KEY)
cfg_list = list_configs(cfg_cpy)
expected_number_of_tests = len(cfg_list) * len(TICKERS)

pbar = tqdm(desc='Running backtests', leave=True, position=1, unit='run', colour='violet', total=expected_number_of_tests)
resulting_GME_df = run_backtest_for_strategy_by_name('Rayner Teo High Winrate')

Running backtests:   0%|          | 0/16800 [00:00<?, ?run/s]

Running `run_backtest_for_strategy_by_name` for {strategy_name} for {len(ticker_list)} tickers
{'ticker': 'ATVI', 'maperiod': range(195, 205), 'rsi_open_period': range(7, 14), 'rsi_close_period': range(27, 35), 'days_ago_close_period': range(7, 13)}
{'ticker': 'EA', 'maperiod': range(195, 205), 'rsi_open_period': range(7, 14), 'rsi_close_period': range(27, 35), 'days_ago_close_period': range(7, 13)}
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 513, in _handle_workers
    cls._maintain_pool(ctx, Process, processes, pool, inqueue,
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 337, in _maintain_pool
    Pool._repopulate_pool_static(ctx, Process, processes, pool,
  File "/usr/lib/python3.8/multiprocessing/pool.p

KeyboardInterrupt: 

In [ ]:
# resulting_GME_df.T[resulting_GME_df.T['Annual return'] == resulting_GME_df.T['Annual return'].max()]
# resulting_GME_df.T[resulting_GME_df.T['Cumulative returns'] == resulting_GME_df.T['Cumulative returns'].max()]
# resulting_GME_df.to_csv('long_run.csv')
# resulting_GME_df.to_csv(f'{DATA_LOCATION}/long_run.csv')
# resulting_GME_df.T.sort_values('Annual return', ascending=False).head(10)
resulting_GME_df

NameError: ignored

In [ ]:
resulting_GME_df.T.columns

Index(['Annual return', 'Cumulative returns', 'Annual volatility',
       'Sharpe ratio', 'Calmar ratio', 'Stability', 'Max drawdown',
       'Omega ratio', 'Sortino ratio', 'Skew', 'Kurtosis', 'Tail ratio',
       'Daily value at risk', 'Gross leverage', 'Daily turnover',
       'days_ago_close_period', 'rsi_close_period', 'rsi_open_period',
       'maperiod'],
      dtype='object')

In [31]:
!ls mount/results

main.csv


In [26]:
!cat mount/results/main.csv | wc -l

In [52]:
!shuf -n 10 mount/results/main.csv

5f94a0a4-a05c-11eb-8ad3-00155d61e157,ATVI,2021-04-18 18:40:16.594367,78.63,14.152655647876955,0.15058240323805255,40.30459153712624,199,7,29,14,12,26,14,8,79.21
20bfbb02-a05c-11eb-a008-00155d61e157,ATVI,2021-04-18 18:38:31.180391,76.15,28.229983119662982,1.287743099557165,16.25461244666545,195,7,29,14,12,26,14,11,76.42
72f67442-a05c-11eb-9513-00155d61e157,ATVI,2021-04-18 18:40:49.112199,51.93,11.403308450736116,0.11455183426454449,21.511192377411664,200,7,30,14,12,26,14,12,52.1
916841f8-a05c-11eb-9513-00155d61e157,ATVI,2021-04-18 18:41:40.189667,78.63,14.152655647876955,0.15058240323805255,40.30459153712624,202,7,32,14,12,26,14,9,79.21
7af94d4a-a05c-11eb-8ad3-00155d61e157,ATVI,2021-04-18 18:41:02.552635,75.81,15.97865259408921,-0.450567336972133,18.999881001347816,200,11,29,14,12,26,14,9,79.21
25c664b6-a05c-11eb-a008-00155d61e157,ATVI,2021-04-18 18:38:39.612688,72.17,28.291727284214506,0.5363112654450405,10.913219616755478,195,9,30,14,12,26,14,11,76.42
b398b442-a05c-11eb-8ad3-00155d61e

In [ ]:
# !rm mount/results/main.csv